In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd

In [ ]:
shp = gpd.GeoDataFrame.from_file('lossdata_footprint/lossdata_footprint.shp')

In [ ]:
#list(shp)

In [ ]:
shp['geometry'].head()

In [ ]:
print(shp['geometry'][0])

In [ ]:
# BBox order
# minimum latitude, minimum longitude, maximum latitude, maximum longitude
# (or South-West-North-East).

In [ ]:
bbox_dict = {}

# iterate over each feature to extract its name and the geometry
for name_tile, polygon in zip(shp['name_tile'], shp['geometry']):

    coords = polygon.exterior.coords[0:4]
    output_list = []

    for pnt in coords:
         x_coord = round(pnt[0])
         y_coord = round(pnt[1])

         output_list.append((x_coord, y_coord))

    min_lat = output_list[0][1]
    min_lon = output_list[0][0]
    max_lat = output_list[2][1]
    max_lon = output_list[2][0]

    bbox_dict[name_tile] = [min_lat, min_lon, max_lat, max_lon]

In [ ]:
for name_key, bbox in bbox_dict.items():

    # we want to query the API with this
    print (name_key, bbox)

In [ ]:
bbox_dict['30N_060W'][0:4]

In [ ]:
def dict_to_df(d):
    df=pd.DataFrame(d.items())
    df.set_index(0, inplace=True)
    return df

In [ ]:
bboxes = pd.DataFrame(list(bbox_dict.items()))

In [ ]:
list(bboxes)

In [ ]:
bboxes.columns = ['tile_key', 'coord']

In [ ]:
bboxes.head()

In [ ]:
bboxes.to_csv('bbox.csv')

http://api.openstreetmap.org/api/0.6

In [ ]:
df = pd.read_csv('bbox.csv')
df.columns = ['index2', 'tile_key', 'coord']

In [67]:
df.head(2)

,index2,tile_key,coord
0,0,20S_170E,"(-30, 170, -20, 180)"
1,1,30S_110E,"(-40, 110, -30, 120)"


In [69]:
len(df['coord'])

504

In [ ]:
import overpass
import csv
import sys
api = overpass.API(timeout=900)

coords = []

with open('bbox.csv') as theFile:
    csvReader = csv.reader(theFile)
    csvReader.__next__()
    for row in csvReader:
	    coords.append(row[2])

for coord in coords:
    api_calls = []
    params = 'node["power"="generator"]'+coord+';out skel;'
	
    try:
        response = api.Get(params)
        #print response
        api_calls.append(response)
        print ('This succeeded: {0}'.format(coord))
    except:
        print ('--------------------------------')
        print ('This failed: {0}'.format(params))
        continue

In [84]:
import json
import requests
import numpy as np
import pandas as pd

In [85]:
table = pd.io.json.json_normalize(response['features'])

In [86]:
list(table)

['geometry.coordinates',
 'geometry.type',
 'id',
 'properties.generator:method',
 'properties.generator:output:electricity',
 'properties.generator:source',
 'properties.name',
 'properties.power',
 'type']

In [87]:
table.isnull().sum()

geometry.coordinates                         0
geometry.type                                0
id                                           0
properties.generator:method                119
properties.generator:output:electricity    119
properties.generator:source                 61
properties.name                            104
properties.power                            60
type                                         0
dtype: int64

In [90]:
table.head()

,geometry.coordinates,geometry.type,id,properties.generator:method,properties.generator:output:electricity,properties.generator:source,properties.name,properties.power,type
0,"(-156.5459296, 20.8056515)",Point,1843489091,NaN,NaN,NaN,NaN,NaN,Feature
1,"(-156.5551839, 20.8224748)",Point,1843489092,NaN,NaN,NaN,NaN,NaN,Feature
2,"(-156.5481612, 20.8099418)",Point,1843489093,NaN,NaN,NaN,NaN,NaN,Feature
3,"(-156.5532359, 20.8194611)",Point,1843489094,NaN,NaN,NaN,NaN,NaN,Feature
4,"(-156.5538428, 20.8205249)",Point,1843489095,NaN,NaN,NaN,NaN,NaN,Feature


In [93]:
table['properties.generator:source'].unique()

array([nan, 'wind', 'coal'], dtype=object)

In [94]:
table.to_csv('nodes_by_bbox_grid.csv')